In [19]:
import numpy as np
import os
import pandas as pd
import pyreadstat as prs
import re

from pathlib import Path

import config as c

In [2]:
print(c.db2018)
for f in sorted(os.listdir(c.db2018)):
    print(f)

../GEOStat/2018-DB_eng/2018 DB
123.sav
conspurch_03.sav
deadmen.sav
familysize.sav
sysschedule.sav
tblavgprices.sav
tblconsumption.sav
tblexpenditures.sav
tblincomes.sav
tblshinda01.sav
tblshinda01_animals.sav
tblshinda01_comfort.sav
tblshinda01_cultures.sav
tblshinda01_lands.sav
tblshinda01_subjects.sav
tblshinda02.sav
tblshinda03_1.sav
tblshinda04.sav
tblshinda04_1.sav


In [3]:
stataDirs = [c.db2018, 
             c.db2017, 
             c.db2016, 
             c.db2015, 
             c.db2014, 
             c.db2013, 
             c.db2012, 
             c.db2011, 
             c.db2010, 
             c.db2009]

for stataDir in stataDirs:
    print('\n')
    print(stataDir)
    cols = 0
    savs = 0
    for f in os.listdir(stataDir):
        if 'deadmen' not in f:
            continue
        filePath = os.path.join(stataDir, f)
        print(f)
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print(df.head())
        #print('count: {} -- {}'.format(len(df.RegNo.unique()), np.sort(df.RegNo.unique()).astype(int)))
        print('number_rows: {}'.format(meta.number_rows))
        print('column_names, n = {}'.format(len(meta.column_names), meta.column_names))
        print('column_labels, n = {}\n'.format(len(meta.column_labels), meta.column_labels))
        colLabels = meta.column_labels
        for l in colLabels:
            print(l)
    break
    print('total savs = {}'.format(savs))
    print('total cols = {}'.format(cols))



../GEOStat/2018-DB_eng/2018 DB
deadmen.sav
        UID  MemberNo  Relations  Gender  BirthYear  Education  Nationality  \
0  321263.0       1.0        1.0     2.0     1937.0        4.0          1.0   
1  321269.0       1.0        1.0     2.0     1937.0        5.0          1.0   
2  321269.0       2.0        2.0     1.0     1940.0        5.0          1.0   
3  321284.0       1.0        1.0     2.0     1944.0        7.0          1.0   
4  321296.0       2.0        2.0     1.0     1940.0        6.0          1.0   

   MaritalStatus  YearsInThisCity  Migration  SpecialStatus1  SpecialStatus2  \
0            1.0              1.0        NaN             NaN             NaN   
1            2.0              5.0        2.0             2.0             9.0   
2            2.0              5.0        2.0             9.0             NaN   
3            1.0              5.0        2.0             2.0             NaN   
4            1.0              5.0        3.0             NaN             NaN   


## Compare all files in a single year

In [4]:
for stataDir in stataDirs:
    n=1
    #print('\n{}'.format(stataDir))
    for f in os.listdir(stataDir):
        #if 'deadmen' not in f:
        #    continue
        filePath = os.path.join(stataDir, f)
        print('#{} {:>5s} {}'.format(n, '', f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('{}'.format('-'*40))
        #print(df.info())
        #print('{}'.format('-'*40))
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        i = 1
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            print('{:<3} {:<4} {:<20}  {}'.format(i, unique_cnt, name, label))
            i+=1
    
        n+=1
        #vvl_dict = meta.variable_value_labels
        #for k1, v1 in vvl_dict.items():
        #    print('  {}  (value type: {})'.format(k1, type(v1)))
        #    for k2, v2 in v1.items():
        #        print('    {}  {}'.format(k2, v2))
            
        print('{}\n'.format('-'*80))
    
    break

#1       tblexpenditures.sav
----------------------------------------
Rows: 11056  Cols: 21
Names: 21  Labels: 21
1   11056 UID                   Unique ID of the household in the quarter
2   5393 sursati__sasmeli__Tambaqo  Expenditures on food, beverages, tobacco (monthly average)
3   562  tansacmeli            Expenditures on clothing and footwear (monthly average)
4   1538 saojaxo               Expenditures on household goods (monthly average)
5   931  samedicino            Expenditures on healthcare (monthly average)
6   2908 energia               Expenditures on fuel and electricity (monthly average)
7   1505 transporti            Expenditures on transport (monthly average)
8   192  ganatleba             Expenditures on education (monthly average)
9   5328 sxva                  Other consumption expenditures (monthly average)
10  11043 samomxmXarjebi        Total cash consumption expenditures (monthly average)
11  541  sasoflo               Expenditures on agriculture (monthly ave

----------------------------------------
Rows: 595992  Cols: 9
Names: 9  Labels: 9
1   11056 UID                   Unique ID of the household in the quarter
2   15   MonthYear             Month and year of operation
3   47   TableNo               Table number from Shinda 04
4   22   ItemNo                Item number from Shinda 04
5   3    Opt                   Option (for TableNo 31 and 32)
6   949  Value                 Cash expenditures or income from selling domestic animals, only for TableNo 226 (GEL); or amount of fuel utilized by household, only for TableNo 331.
7   87   raodenoba             Quantity of purchased products (TableNo 101). Quantity of purchased (TableNo 223) or sold (TableNo 226) domestic animals
8   300  raodenobaufaso        Estimated expenditures on the products received free of charge. For TableNo 101 - quantity of products received free of charge
9   56   ufaso101              Estimated expenditures on the products received free of charge (only for TableNo 10

## Compare a single file across all years

In [5]:
specific_file = 'sysschedule.sav'
for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    for f in os.listdir(stataDir):
        if specific_file not in f:
            continue
            
        filePath = os.path.join(stataDir, f)
        print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        print(df.head())
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('{}'.format('-'*40))
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        i = 1
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            print('{:<3} {:<6} {:<20}  {}'.format(i, unique_cnt, name, label))
            i+=1
        
        print('\n---- Fields with codes, and their lookup values ---')
        vvl_dict = meta.variable_value_labels
        for k1, v1 in vvl_dict.items():
            print('{:<5}'.format(k1, type(v1)))
            for k2, v2 in v1.items():
                print('  {:<5} {}'.format(k2, v2))
            
        print('{}\n'.format('-'*80))
        break

2018  sysschedule.sav
      MonthYear  QuartNo       UID   DiaryID  RegNo  UrbanOrRural     Weights
0  1.373414e+10     87.0  321260.0  107549.0    0.0           1.0  135.234085
1  1.373414e+10     87.0  321261.0  107550.0    0.0           1.0  135.234085
2  1.373414e+10     87.0  321262.0  107551.0    0.0           1.0  135.234085
3  1.373414e+10     87.0  321263.0  107552.0    0.0           1.0  135.234085
4  1.373414e+10     87.0  321264.0  107553.0    0.0           1.0  135.234085
----------------------------------------
Rows: 11056  Cols: 7
Names: 7  Labels: 7
1   12     MonthYear             Month and Year of Interview
2   4      QuartNo               Quarter ID
3   11056  UID                   Unique ID of the household in the quarter
4   6497   DiaryID               Unique ID of the household during the whole period
5   10     RegNo                 Region
6   2      UrbanOrRural          Urban Or Rural
7   197    Weights               Sampling weights of the household

---- Fie

4   9956   DiaryID               Unique ID of the household during the whole period
5   10     RegNo                 Region
6   2      UrbanOrRural          Urban Or Rural
7   454    Weights               Sampling weights of the household

---- Fields with codes, and their lookup values ---
RegNo
  0.0   Kakheti
  1.0   Tbilisi
  2.0   Shida Kartli
  3.0   Kvemo Kartli
  5.0   Samtskhe-Javakheti
  7.0   Adjara A.R.
  8.0   Guria
  9.0   Samegrelo-Zemo Svaneti
  10.0  Imereti, Racha-Lechkhumi and Kvemo Svaneti
  11.0  Mtskheta-Mtianeti
UrbanOrRural
  1.0   Urban
  2.0   Rural
--------------------------------------------------------------------------------

2009  sysschedule.sav
      MonthYear  QuartNo       UID  DiaryID  RegNo  UrbanOrRural    Weights
0  1.345015e+10     51.0  180569.0  53306.0    0.0           1.0  352.70353
1  1.345015e+10     51.0  180570.0  53308.0    0.0           1.0  352.70353
2  1.345015e+10     51.0  180571.0  53309.0    0.0           1.0  352.70353
3  1.34501

## Check number of unique regions and their values

In [6]:
specific_file = 'sysschedule.sav'
for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    for f in os.listdir(stataDir):
        if specific_file not in f:
            continue
            
        filePath = os.path.join(stataDir, f)
        print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))
        
        regions = np.sort(df.RegNo.unique()).astype(int)
        print('Count: {} -- Region Values: {}'.format(len(regions), regions))
        print('{}\n'.format('-'*80))
    
    #break

2018  sysschedule.sav
Rows: 11056  Cols: 7
Count: 10 -- Region Values: [ 0  1  2  3  5  7  8  9 10 11]
--------------------------------------------------------------------------------

2017  sysschedule.sav
Rows: 11590  Cols: 7
Count: 10 -- Region Values: [ 0  1  2  3  5  7  8  9 10 11]
--------------------------------------------------------------------------------

2016  sysschedule.sav
Rows: 10858  Cols: 7
Count: 10 -- Region Values: [ 0  1  2  3  5  7  8  9 10 11]
--------------------------------------------------------------------------------

2015  sysschedule.sav
Rows: 10999  Cols: 7
Count: 10 -- Region Values: [ 0  1  2  3  5  7  8  9 10 11]
--------------------------------------------------------------------------------

2014  sysschedule.sav
Rows: 11165  Cols: 7
Count: 10 -- Region Values: [ 0  1  2  3  5  7  8  9 10 11]
--------------------------------------------------------------------------------

2013  sysschedule.sav
Rows: 11102  Cols: 7
Count: 10 -- Region Values: [ 0 

## Get all questions (long field names) for 2018

In [51]:
search_year = 2018
i = 1
outCsv = os.path.join(c.outDir, 'fields.csv')
sep = '\t'

try: os.remove(outCsv)
except: Path(outCsv).touch()

for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    if int(year) != search_year:
        continue
    print('yr: {}   I want: {}'.format(year, search_year))

    for f in sorted(os.listdir(stataDir)):
        #if specific_file not in f:
        #    continue
        fname = f.strip('.sav')
        filePath = os.path.join(stataDir, f)
        #print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        #print('{}'.format('-'*40))
        #print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        #print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        
        header = 'Number{0}Year{0}sav file{0}Count of unique values{0}Field (short){0}Field (long)\n'.format(sep)
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            info_string = '{:<2}  {:<4}  {:<6}  {:<6}  {:<20}  {}'.format(i, year, fname, unique_cnt, name, label)
            csv_string = '{1}{0}{2}{0}{3}{0}{4}{0}{5}{0}{6}'.format(sep, i, year, fname, unique_cnt, name, label)
            print(csv_string)
            with open(outCsv, 'a') as f:
                if i==1:
                    f.write(header)
                f.write(csv_string + '\n')
            i+=1
            
        """
        print('\n---- Fields with codes, and their lookup values ---')
        vvl_dict = meta.variable_value_labels
        for k1, v1 in vvl_dict.items():
            print('{:<5}'.format(k1, type(v1)))
            for k2, v2 in v1.items():
                print('  {:<5} {}'.format(k2, v2))
        """    
        print('{}\n'.format('-'*80))
        #break

yr: 2018   I want: 2018
1	2018	123	90	QuartNo	Quarter ID
2	2018	123	4	Quarters	Quarter in a year
3	2018	123	23	YarNo	Year
4	2018	123	90	YearQuart	Quarter and year
--------------------------------------------------------------------------------

5	2018	conspurch_03	10924	UID	Unique ID of the household in the quarter
6	2018	conspurch_03	192	ProductCode	Product code (COICOP)
7	2018	conspurch_03	1242	Quantity	Quantity of food products purchased for own consumption (KG/L average per month)
8	2018	conspurch_03	756	Paid	How much was paid in GEL (average per month)
9	2018	conspurch_03	4763	Prices	Price of the product in GEL (average per month)
--------------------------------------------------------------------------------

10	2018	deadmen	216	UID	Unique ID of the household in the quarter
11	2018	deadmen	7	MemberNo	Household member number from Shinda 02
12	2018	deadmen	8	Relations	Relationship to the head of household
13	2018	deadmen	2	Gender	Gender
14	2018	deadmen	53	BirthYear	Birth Year
15	2

147	2018	tblshinda02	11056	UID	Unique ID of the household in the quarter
148	2018	tblshinda02	14	MemberNo	Household member number
149	2018	tblshinda02	11	Relations	Relationship to the head of household
150	2018	tblshinda02	2	Gender	Gender
151	2018	tblshinda02	13	Education	Achieved level of education
152	2018	tblshinda02	9	Nationality	Ethnicity
153	2018	tblshinda02	5	MaritalStatus	Marital status
154	2018	tblshinda02	5	YearsInThisCity	How long have you lived at this address?
155	2018	tblshinda02	4	Migration	If moved, from where?
156	2018	tblshinda02	8	SpecialStatus1	Special status 1
157	2018	tblshinda02	7	SpecialStatus2	Special status 2
158	2018	tblshinda02	5	SpecialStatus3	Special status 3
159	2018	tblshinda02	6	NotPresentNow	Temporarily (from 3 months to a year) absent household member
160	2018	tblshinda02	8	Changes	Changes in household composition compared to the 1st interview
161	2018	tblshinda02	4	Insurance	Do you have health insurance?
162	2018	tblshinda02	8	InsuranceType	Health in